In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

pd.options.display.max_colwidth = 100

In [2]:
df = pd.read_csv("tweet_covid19.csv")

In [3]:
df.columns

Index(['id', 'date', 'time', 'timezone', 'username', 'name', 'tweet',
       'mentions', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags'],
      dtype='object')

In [4]:
df

,id,date,time,timezone,username,name,tweet,mentions,replies_count,retweets_count,likes_count,hashtags
0,1233996577276076032,2020-03-01,09:34:45,Iran Standard Time,elijah1334,Elijah Okemwa,Ghanaians. How come #coronavirus isn't yet in your country. #covid19 is on rampage across the wo...,[],0,0,0,"['#coronavirus', '#covid19', '#thankyoublacknana', '#coronavirusinkenya', '#coronavirusupdates']"
1,1233996572653953024,2020-03-01,09:34:44,Iran Standard Time,steffena380,"Steffen, 👨🏼‍🦳✈️👨🏼‍✈️",Einfach nicht so viel\nNudeln essen...\n\nDann braucht Ihr auch nicht das ganze Klopapier zu Ham...,[],0,0,0,['#covid19']
2,1233996566626816000,2020-03-01,09:34:42,Iran Standard Time,dileepvandanapu,DILEEPV,"To avoid #COVID19 spread, there can be a mobile app that detects the distance of any other mobil...",[],0,0,0,['#covid19']
3,1233996566626652162,2020-03-01,09:34:42,Iran Standard Time,rezaram65736521,Reza Ramdani,"Itu menunjukkan pemerintah Indonesia sangat kreatif & care kepada warganya, hanya warga asing yg...",['alvinlie21'],0,0,0,[]
4,1233996561295671296,2020-03-01,09:34:41,Iran Standard Time,nya_kaeru,にゃーと鳴くかえる,患者a→妻→女児かな\n家庭内に入るとアッサリ感染するね...\n#COVID19 \n\n大阪府／報道発表資料／新型コロナウイルス感染症患者の発生について http://www.pref....,[],0,0,0,['#covid19']
...,...,...,...,...,...,...,...,...,...,...,...,...
41621,1233777975813910528,2020-02-29,19:06:06,Iran Standard Time,poisky71,Pois’Ky 💿📀,Le Coronavirus va t’il nous atteindre ? Dans se monde où là sciences dépassent nos attentes.. \n...,[],0,0,0,"['#coronavirus', '#covid19', '#coronavid19', '#tristemonde']"
41622,1233777973075075072,2020-02-29,19:06:05,Iran Standard Time,amstanley,malcolm,Canadian paper has the local angle on #COVID19 https://twitter.com/davidjuurlink/status/1233735...,[],0,0,0,['#covid19']
41623,1233777971586007040,2020-02-29,19:06:05,Iran Standard Time,annette0811,Anna Celina,Imagino q le han hecho estudios para descartar el covid19 ya q el está muy expuesto saludando a ...,['drclaudiosalud'],0,0,1,[]
41624,1233777964950708227,2020-02-29,19:06:03,Iran Standard Time,wendydavitto,Wonderful Lady,@realDonaldTrump What are you hiding about the #COVID19 ??? Why won't you let the MEDICAL exper...,"['realdonaldtrump', 'drnancym_cdc']",0,0,0,"['#covid19', '#covid19us', '#hoax']"


In [5]:
# find number of missing data based on each column
missing_data = df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print("\n")

id
False    41626
Name: id, dtype: int64


date
False    41626
Name: date, dtype: int64


time
False    41626
Name: time, dtype: int64


timezone
False    41626
Name: timezone, dtype: int64


username
False    41626
Name: username, dtype: int64


name
False    41626
Name: name, dtype: int64


tweet
False    41626
Name: tweet, dtype: int64


mentions
False    41626
Name: mentions, dtype: int64


replies_count
False    41626
Name: replies_count, dtype: int64


retweets_count
False    41626
Name: retweets_count, dtype: int64


likes_count
False    41626
Name: likes_count, dtype: int64


hashtags
False    41626
Name: hashtags, dtype: int64




In [6]:
tweets = df[['tweet']].copy()
tweets = tweets.rename({'tweet': 'SentimentText'}, axis=1)
tweets

,SentimentText
0,Ghanaians. How come #coronavirus isn't yet in your country. #covid19 is on rampage across the wo...
1,Einfach nicht so viel\nNudeln essen...\n\nDann braucht Ihr auch nicht das ganze Klopapier zu Ham...
2,"To avoid #COVID19 spread, there can be a mobile app that detects the distance of any other mobil..."
3,"Itu menunjukkan pemerintah Indonesia sangat kreatif & care kepada warganya, hanya warga asing yg..."
4,患者a→妻→女児かな\n家庭内に入るとアッサリ感染するね...\n#COVID19 \n\n大阪府／報道発表資料／新型コロナウイルス感染症患者の発生について http://www.pref....
...,...
41621,Le Coronavirus va t’il nous atteindre ? Dans se monde où là sciences dépassent nos attentes.. \n...
41622,Canadian paper has the local angle on #COVID19 https://twitter.com/davidjuurlink/status/1233735...
41623,Imagino q le han hecho estudios para descartar el covid19 ya q el está muy expuesto saludando a ...
41624,@realDonaldTrump What are you hiding about the #COVID19 ??? Why won't you let the MEDICAL exper...


In [7]:
tweets.isna().sum()

SentimentText    0
dtype: int64

In [8]:
rand_indexs = np.random.randint(1,len(tweets),50).tolist()
tweets["SentimentText"][rand_indexs]

8260     Es muy importante desinfectar los artículos de uso diario como celulares y tabletas. #Coronaviru...
33210    Fear As unknown Man Sneezed and Coughed at The Atm Queue that Made Everybody to Run Away.  http:...
31638                                                                                         Where is that?
35509    #JAGUARESxESPN #nochedebrujas #UnDiaMasPara #quesoblanco #FelizSabado #perpinya #COVID19 #29DeFe...
6155     Also... i’m more worried about not getting covid19 right now and the economy crashing and losing...
32160    Giornalisti che erano alla conferenza stampa @DPCgov mi riferiscono che oggi non sono state dist...
28574    Dass ausgerechnet Ihr Euch vor dem #COVID19 fürchtet wundert mich. Der hat nämlich im Gegensatz ...
25152    Coronavirus 2019-nCoV (COVID19) outbreak\n\n#COVID19 has now spread to over 60 countries (reache...
1465     Kind hearted Tzuyu donate to S.Korea and China. We forever remember the Taiwan flag issue, C-net...
7357     Of the Pop

In [9]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    text = re.sub(r'\b\w{1,3}\b', '', text)
    text = text.replace('\n', ' ')
    
    return text

tweets['Tweet_punct'] = tweets['SentimentText'].apply(lambda x: remove_punct(x))
tweets.head(10)

,SentimentText,Tweet_punct
0,Ghanaians. How come #coronavirus isn't yet in your country. #covid19 is on rampage across the wo...,Ghanaians come coronavirus isnt your country covid rampage across world your country Thi...
1,Einfach nicht so viel\nNudeln essen...\n\nDann braucht Ihr auch nicht das ganze Klopapier zu Ham...,Einfach nicht viel Nudeln essen Dann braucht auch nicht ganze Klopapier Hamstern COVID
2,"To avoid #COVID19 spread, there can be a mobile app that detects the distance of any other mobil...",avoid COVID spread there mobile that detects distance other mobile coming closer notify...
3,"Itu menunjukkan pemerintah Indonesia sangat kreatif & care kepada warganya, hanya warga asing yg...",menunjukkan pemerintah Indonesia sangat kreatif care kepada warganya hanya warga asing diperk...
4,患者a→妻→女児かな\n家庭内に入るとアッサリ感染するね...\n#COVID19 \n\n大阪府／報道発表資料／新型コロナウイルス感染症患者の発生について http://www.pref....,→→女児かな 家庭内に入るとアッサリ感染するね COVID ／報道発表資料／新型コロナウイルス感染症患者の発生について httpwwwprefosakalgjphodoindexphps...
5,今思えば空気清浄機の #新型コロナウィルス 対応製品が出たらヤバいことになるが命がけだよな… #COVID19 https://twitter.com/sharp_jp/status/123...,今思えば空気清浄機の 新型コロナウィルス 対応製品が出たらヤバいことになるが命がけだよな… COVID httpstwittercomsharpjpstatus …
6,"If anything good comes out of COVID19, it'll be that even capitalists will see the need for shor...",anything good comes COVID itll that even capitalists will need short term disability pai...
7,Incredible. Another tweet in this thread says some S Koreans visited Milan & are #COVID19 positi...,Incredible Another tweet this thread says some Koreans visited Milan COVID positive 😲 https...
8,サニーレタスの値札が、なんと「本体1975円」になっている!!\n#COVID19 の影響でこんなことになっているの？と思いましたが、棚のポップは「本体198円」になっているから、値札シールの...,サニーレタスの値札が、「」になっている COVID の影響でこんなことになっているの？と思いましたが、棚のポップは「」になっているから、値札シールの張り間違えとは思いますが。。。 店員さん、い...
9,I recall Peter Duesberg lecturing us at UC Berkeley on how HIV didn't cause AIDS! (made me angry...,recall Peter Duesberg lecturing Berkeley didnt cause AIDS made angry very glad that PO...


In [10]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

tweets['Tweet_tokenized'] = tweets['Tweet_punct'].apply(lambda x: tokenization(x.lower()))
tweets.head()

,SentimentText,Tweet_punct,Tweet_tokenized
0,Ghanaians. How come #coronavirus isn't yet in your country. #covid19 is on rampage across the wo...,Ghanaians come coronavirus isnt your country covid rampage across world your country Thi...,"[ghanaians, come, coronavirus, isnt, your, country, covid, rampage, across, world, your, country..."
1,Einfach nicht so viel\nNudeln essen...\n\nDann braucht Ihr auch nicht das ganze Klopapier zu Ham...,Einfach nicht viel Nudeln essen Dann braucht auch nicht ganze Klopapier Hamstern COVID,"[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganze, klopapier, hamstern, co..."
2,"To avoid #COVID19 spread, there can be a mobile app that detects the distance of any other mobil...",avoid COVID spread there mobile that detects distance other mobile coming closer notify...,"[, avoid, covid, spread, there, mobile, that, detects, distance, other, mobile, coming, closer, ..."
3,"Itu menunjukkan pemerintah Indonesia sangat kreatif & care kepada warganya, hanya warga asing yg...",menunjukkan pemerintah Indonesia sangat kreatif care kepada warganya hanya warga asing diperk...,"[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as..."
4,患者a→妻→女児かな\n家庭内に入るとアッサリ感染するね...\n#COVID19 \n\n大阪府／報道発表資料／新型コロナウイルス感染症患者の発生について http://www.pref....,→→女児かな 家庭内に入るとアッサリ感染するね COVID ／報道発表資料／新型コロナウイルス感染症患者の発生について httpwwwprefosakalgjphodoindexphps...,"[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph..."


In [11]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    t = [word for word in text if word not in stopword]
    if len(t)>3:
        text = t
    return text
    
tweets['Tweet_nonstop'] = tweets['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
tweets.head(10)

,SentimentText,Tweet_punct,Tweet_tokenized,Tweet_nonstop
0,Ghanaians. How come #coronavirus isn't yet in your country. #covid19 is on rampage across the wo...,Ghanaians come coronavirus isnt your country covid rampage across world your country Thi...,"[ghanaians, come, coronavirus, isnt, your, country, covid, rampage, across, world, your, country...","[ghanaians, come, coronavirus, isnt, country, covid, rampage, across, world, country, crazy, cra..."
1,Einfach nicht so viel\nNudeln essen...\n\nDann braucht Ihr auch nicht das ganze Klopapier zu Ham...,Einfach nicht viel Nudeln essen Dann braucht auch nicht ganze Klopapier Hamstern COVID,"[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganze, klopapier, hamstern, co...","[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganze, klopapier, hamstern, co..."
2,"To avoid #COVID19 spread, there can be a mobile app that detects the distance of any other mobil...",avoid COVID spread there mobile that detects distance other mobile coming closer notify...,"[, avoid, covid, spread, there, mobile, that, detects, distance, other, mobile, coming, closer, ...","[, avoid, covid, spread, mobile, detects, distance, mobile, coming, closer, notify, person, main..."
3,"Itu menunjukkan pemerintah Indonesia sangat kreatif & care kepada warganya, hanya warga asing yg...",menunjukkan pemerintah Indonesia sangat kreatif care kepada warganya hanya warga asing diperk...,"[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as...","[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as..."
4,患者a→妻→女児かな\n家庭内に入るとアッサリ感染するね...\n#COVID19 \n\n大阪府／報道発表資料／新型コロナウイルス感染症患者の発生について http://www.pref....,→→女児かな 家庭内に入るとアッサリ感染するね COVID ／報道発表資料／新型コロナウイルス感染症患者の発生について httpwwwprefosakalgjphodoindexphps...,"[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph...","[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph..."
5,今思えば空気清浄機の #新型コロナウィルス 対応製品が出たらヤバいことになるが命がけだよな… #COVID19 https://twitter.com/sharp_jp/status/123...,今思えば空気清浄機の 新型コロナウィルス 対応製品が出たらヤバいことになるが命がけだよな… COVID httpstwittercomsharpjpstatus …,"[今思えば空気清浄機の, 新型コロナウィルス, 対応製品が出たらヤバいことになるが命がけだよな, covid, httpstwittercomsharpjpstatus, ]","[今思えば空気清浄機の, 新型コロナウィルス, 対応製品が出たらヤバいことになるが命がけだよな, covid, httpstwittercomsharpjpstatus, ]"
6,"If anything good comes out of COVID19, it'll be that even capitalists will see the need for shor...",anything good comes COVID itll that even capitalists will need short term disability pai...,"[, anything, good, comes, covid, itll, that, even, capitalists, will, need, short, term, disabil...","[, anything, good, comes, covid, itll, even, capitalists, need, short, term, disability, paid, l..."
7,Incredible. Another tweet in this thread says some S Koreans visited Milan & are #COVID19 positi...,Incredible Another tweet this thread says some Koreans visited Milan COVID positive 😲 https...,"[incredible, another, tweet, this, thread, says, some, koreans, visited, milan, covid, positive,...","[incredible, another, tweet, thread, says, koreans, visited, milan, covid, positive, httpstwitte..."
8,サニーレタスの値札が、なんと「本体1975円」になっている!!\n#COVID19 の影響でこんなことになっているの？と思いましたが、棚のポップは「本体198円」になっているから、値札シールの...,サニーレタスの値札が、「」になっている COVID の影響でこんなことになっているの？と思いましたが、棚のポップは「」になっているから、値札シールの張り間違えとは思いますが。。。 店員さん、い...,"[サニーレタスの値札が, になっている, covid, の影響でこんなことになっているの, と思いましたが, 棚のポップは, になっているから, 値札シールの張り間違えとは思いますが, 店員さ...","[サニーレタスの値札が, になっている, covid, の影響でこんなことになっているの, と思いましたが, 棚のポップは, になっているから, 値札シールの張り間違えとは思いますが, 店員さ..."
9,I recall Peter Duesberg lecturing us at UC Berkeley on how HIV didn't cause AIDS! (made me angry...,recall Peter Duesberg lecturing Berkeley didnt cause AIDS made angry very glad that PO...,"[, recall, peter, duesberg, lecturing, berkeley, didnt, cause, aids, made, angry, very, glad, th...","[, recall, peter, duesberg, lecturing, berkeley, didnt, cause, aids, made, angry

In [12]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

tweets['Tweet_stemmed'] = tweets['Tweet_nonstop'].apply(lambda x: stemming(x))
tweets.head()

,SentimentText,Tweet_punct,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed
0,Ghanaians. How come #coronavirus isn't yet in your country. #covid19 is on rampage across the wo...,Ghanaians come coronavirus isnt your country covid rampage across world your country Thi...,"[ghanaians, come, coronavirus, isnt, your, country, covid, rampage, across, world, your, country...","[ghanaians, come, coronavirus, isnt, country, covid, rampage, across, world, country, crazy, cra...","[ghanaian, come, coronaviru, isnt, countri, covid, rampag, across, world, countri, crazi, crazi,..."
1,Einfach nicht so viel\nNudeln essen...\n\nDann braucht Ihr auch nicht das ganze Klopapier zu Ham...,Einfach nicht viel Nudeln essen Dann braucht auch nicht ganze Klopapier Hamstern COVID,"[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganze, klopapier, hamstern, co...","[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganze, klopapier, hamstern, co...","[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganz, klopapi, hamstern, covid]"
2,"To avoid #COVID19 spread, there can be a mobile app that detects the distance of any other mobil...",avoid COVID spread there mobile that detects distance other mobile coming closer notify...,"[, avoid, covid, spread, there, mobile, that, detects, distance, other, mobile, coming, closer, ...","[, avoid, covid, spread, mobile, detects, distance, mobile, coming, closer, notify, person, main...","[, avoid, covid, spread, mobil, detect, distanc, mobil, come, closer, notifi, person, maintain, ..."
3,"Itu menunjukkan pemerintah Indonesia sangat kreatif & care kepada warganya, hanya warga asing yg...",menunjukkan pemerintah Indonesia sangat kreatif care kepada warganya hanya warga asing diperk...,"[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as...","[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as...","[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as..."
4,患者a→妻→女児かな\n家庭内に入るとアッサリ感染するね...\n#COVID19 \n\n大阪府／報道発表資料／新型コロナウイルス感染症患者の発生について http://www.pref....,→→女児かな 家庭内に入るとアッサリ感染するね COVID ／報道発表資料／新型コロナウイルス感染症患者の発生について httpwwwprefosakalgjphodoindexphps...,"[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph...","[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph...","[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph..."


In [13]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

tweets['Tweet_lemmatized'] = tweets['Tweet_nonstop'].apply(lambda x: lemmatizer(x))
tweets.head()

,SentimentText,Tweet_punct,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed,Tweet_lemmatized
0,Ghanaians. How come #coronavirus isn't yet in your country. #covid19 is on rampage across the wo...,Ghanaians come coronavirus isnt your country covid rampage across world your country Thi...,"[ghanaians, come, coronavirus, isnt, your, country, covid, rampage, across, world, your, country...","[ghanaians, come, coronavirus, isnt, country, covid, rampage, across, world, country, crazy, cra...","[ghanaian, come, coronaviru, isnt, countri, covid, rampag, across, world, countri, crazi, crazi,...","[ghanaians, come, coronavirus, isnt, country, covid, rampage, across, world, country, crazy, cra..."
1,Einfach nicht so viel\nNudeln essen...\n\nDann braucht Ihr auch nicht das ganze Klopapier zu Ham...,Einfach nicht viel Nudeln essen Dann braucht auch nicht ganze Klopapier Hamstern COVID,"[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganze, klopapier, hamstern, co...","[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganze, klopapier, hamstern, co...","[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganz, klopapi, hamstern, covid]","[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganze, klopapier, hamstern, co..."
2,"To avoid #COVID19 spread, there can be a mobile app that detects the distance of any other mobil...",avoid COVID spread there mobile that detects distance other mobile coming closer notify...,"[, avoid, covid, spread, there, mobile, that, detects, distance, other, mobile, coming, closer, ...","[, avoid, covid, spread, mobile, detects, distance, mobile, coming, closer, notify, person, main...","[, avoid, covid, spread, mobil, detect, distanc, mobil, come, closer, notifi, person, maintain, ...","[, avoid, covid, spread, mobile, detects, distance, mobile, coming, closer, notify, person, main..."
3,"Itu menunjukkan pemerintah Indonesia sangat kreatif & care kepada warganya, hanya warga asing yg...",menunjukkan pemerintah Indonesia sangat kreatif care kepada warganya hanya warga asing diperk...,"[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as...","[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as...","[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as...","[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as..."
4,患者a→妻→女児かな\n家庭内に入るとアッサリ感染するね...\n#COVID19 \n\n大阪府／報道発表資料／新型コロナウイルス感染症患者の発生について http://www.pref....,→→女児かな 家庭内に入るとアッサリ感染するね COVID ／報道発表資料／新型コロナウイルス感染症患者の発生について httpwwwprefosakalgjphodoindexphps...,"[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph...","[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph...","[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph...","[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph..."


In [14]:
# We are gonna find what emoticons are used in our dataset

tweets_text = tweets.SentimentText.str.cat()
emos = set(re.findall(r" ([xX:;][-']?.) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
sorted(emos_count,reverse=True)

[(16940, ':/'),
 (5750, ': '),
 (469, 'xa'),
 (110, 'XI'),
 (103, 'Xi'),
 (95, 'xx'),
 (62, 'xA'),
 (62, 'XM'),
 (54, ':)'),
 (45, 'xq'),
 (44, 'Xu'),
 (42, 'xD'),
 (30, ':('),
 (24, 'X,'),
 (22, 'X.'),
 (20, ';)'),
 (7, '::'),
 (7, ':-)'),
 (5, ';-)'),
 (4, ':-('),
 (2, ':<'),
 (1, 'x%'),
 (1, ';-('),
 (1, ':…'),
 (1, ':v')]

In [15]:
HAPPY_EMO = r" ([xX;:]-?[dD)]|:-?[\)]|[;:][pP]) "
SAD_EMO = r" (:'?[/|\(]) "
print("Happy emoticons:", set(re.findall(HAPPY_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(SAD_EMO, tweets_text)))

Happy emoticons: {':-)', ';)', 'xD', ':)', ';-)'}
Sad emoticons: {':/', ':('}


In [16]:
# Uncomment this line if you haven't downloaded punkt before
# or just run it as it is and uncomment it if you got an error.
# nltk.download('punkt')

def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

In [17]:
# nltk.download("words")
words = set(nltk.corpus.words.words())

test_text_data = " ".join(w for w in nltk.wordpunct_tokenize(tweets.Tweet_punct.str.cat()) \
         if w.lower() in words or not w.isalpha())

In [18]:
most_used_words(test_text_data)[:10]

There is 21431 different words


['COVID', '…', '’', 'covid', 'that', 'from', 'Covid', 'this', 'have', 'with']

In [19]:
# nltk.download("stopwords")
stopword = nltk.corpus.stopwords.words('english')
mw = most_used_words(tweets.Tweet_punct.str.cat())
most_words = []
for w in mw:
    if len(most_words) == 1000:
        break
    if w in stopword:
        continue
    else:
        most_words.append(w)

There is 124295 different words


In [20]:
# What we did is to filter only non stop words.
# We will now get a look to the top 1000 words
sorted(most_words)

['AUTOMEDIQUES',
 'Acude',
 'Afghanistan',
 'Algunos',
 'Also',
 'America',
 'American',
 'Americans',
 'Australia',
 'BreakingNews',
 'CDCgov',
 'CORONAVIRUS',
 'COVID',
 'COVIDCOVID',
 'COVIDNigeria',
 'COVIDUS',
 'COVIDfrance',
 'COVIDitalia',
 'COVIDmx',
 'COVIDー',
 'California',
 'Canada',
 'Cases',
 'CataAndramuno',
 'China',
 'ChinaCoronaVirus',
 'Chinese',
 'CoVid',
 'Coahuila',
 'Confirmed',
 'Corona',
 'CoronaVirus',
 'CoronaVirusCanada',
 'CoronaVirusDE',
 'CoronaVirusUpdate',
 'CoronaVirusUpdates',
 'Coronavid',
 'Coronavirus',
 'CoronavirusFrance',
 'CoronavirusKe',
 'CoronavirusOutbreak',
 'Coronavirusitalia',
 'Coronavirusmexico',
 'Coronavirustruth',
 'Costco',
 'County',
 'Covid',
 'Covidusa',
 'Death',
 'Deaths',
 'Democrats',
 'Dems',
 'Department',
 'Disease',
 'Donald',
 'Dont',
 'Ebola',
 'Ecuador',
 'Epidemic',
 'España',
 'Europe',
 'Even',
 'FakeNews',
 'Fauci',
 'February',
 'First',
 'France',
 'From',
 'Germany',
 'Good',
 'Guayaquil',
 'Have',
 'Health',
 '

In [21]:
# I'm defining this function to use it in the 
# Data Preparation Phase
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# nltk.download('wordnet')
def stem_tokenize(text):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(text)]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]

In [22]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt


text = " ".join(tweet for tweet in tweets.Tweet_punct)
stopwords = set(STOPWORDS)
wordcloud = WordCloud(max_font_size=50, max_words=100,stopwords= stopwords ,  background_color="white").generate(text)

plt.figure(figsize=(100,50))
plt.tight_layout()
plt.axis("off")
wordcloud.to_file('tweets_covid19.png')

plt.imshow(wordcloud, interpolation="bilinear")
plt.show()


<Figure size 10000x5000 with 1 Axes>

In [23]:
from textblob import TextBlob

In [24]:
tweets['polarity'] = tweets.Tweet_punct.apply(lambda text : TextBlob(text).sentiment.polarity)
tweets['subjectivity'] = tweets.Tweet_punct.apply(lambda text : TextBlob(text).sentiment.subjectivity)

In [25]:
tweets

,SentimentText,Tweet_punct,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed,Tweet_lemmatized,polarity,subjectivity
0,Ghanaians. How come #coronavirus isn't yet in your country. #covid19 is on rampage across the wo...,Ghanaians come coronavirus isnt your country covid rampage across world your country Thi...,"[ghanaians, come, coronavirus, isnt, your, country, covid, rampage, across, world, your, country...","[ghanaians, come, coronavirus, isnt, country, covid, rampage, across, world, country, crazy, cra...","[ghanaian, come, coronaviru, isnt, countri, covid, rampag, across, world, countri, crazi, crazi,...","[ghanaians, come, coronavirus, isnt, country, covid, rampage, across, world, country, crazy, cra...",-0.233333,0.766667
1,Einfach nicht so viel\nNudeln essen...\n\nDann braucht Ihr auch nicht das ganze Klopapier zu Ham...,Einfach nicht viel Nudeln essen Dann braucht auch nicht ganze Klopapier Hamstern COVID,"[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganze, klopapier, hamstern, co...","[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganze, klopapier, hamstern, co...","[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganz, klopapi, hamstern, covid]","[einfach, nicht, viel, nudeln, essen, dann, braucht, auch, nicht, ganze, klopapier, hamstern, co...",0.000000,0.000000
2,"To avoid #COVID19 spread, there can be a mobile app that detects the distance of any other mobil...",avoid COVID spread there mobile that detects distance other mobile coming closer notify...,"[, avoid, covid, spread, there, mobile, that, detects, distance, other, mobile, coming, closer, ...","[, avoid, covid, spread, mobile, detects, distance, mobile, coming, closer, notify, person, main...","[, avoid, covid, spread, mobil, detect, distanc, mobil, come, closer, notifi, person, maintain, ...","[, avoid, covid, spread, mobile, detects, distance, mobile, coming, closer, notify, person, main...",-0.125000,0.375000
3,"Itu menunjukkan pemerintah Indonesia sangat kreatif & care kepada warganya, hanya warga asing yg...",menunjukkan pemerintah Indonesia sangat kreatif care kepada warganya hanya warga asing diperk...,"[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as...","[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as...","[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as...","[, menunjukkan, pemerintah, indonesia, sangat, kreatif, care, kepada, warganya, hanya, warga, as...",0.000000,0.000000
4,患者a→妻→女児かな\n家庭内に入るとアッサリ感染するね...\n#COVID19 \n\n大阪府／報道発表資料／新型コロナウイルス感染症患者の発生について http://www.pref....,→→女児かな 家庭内に入るとアッサリ感染するね COVID ／報道発表資料／新型コロナウイルス感染症患者の発生について httpwwwprefosakalgjphodoindexphps...,"[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph...","[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph...","[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph...","[, 女児かな, 家庭内に入るとアッサリ感染するね, covid, 報道発表資料, 新型コロナウイルス感染症患者の発生について, httpwwwprefosakalgjphodoindexph...",0.000000,0.000000
...,...,...,...,...,...,...,...,...
41621,Le Coronavirus va t’il nous atteindre ? Dans se monde où là sciences dépassent nos attentes.. \n...,Coronavirus ’ nous atteindre Dans monde sciences dépassent attentes Réalité fiction Cr...,"[, coronavirus, nous, atteindre, dans, monde, sciences, dépassent, attentes, réalité, fiction, c...","[, coronavirus, nous, atteindre, dans, monde, sciences, dépassent, attentes, réalité, fiction, c...","[, coronaviru, nou, atteindr, dan, mond, scienc, dépassent, attent, réalité, fiction, croit, con...","[, coronavirus, nous, atteindre, dans, monde, science, dépassent, attentes, réalité, fiction, cr...",0.000000,0.000000
41622,Canadian paper has the local angle on #COVID19 https://twitter.com/davidjuurlink/status/1233735...,Canadian paper local

In [26]:
tweets.Sentiment.hist()

AttributeError: 'DataFrame' object has no attribute 'Sentiment'